In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [48]:
def point_vizual(point):#на вход принимает словaрь с команда:очки
    point_df = pd.DataFrame({
        'счет': list(point.values())
    })
    point_df.index = list(d.keys())
    point_df.sort_values(by="счет",axis=0,ascending=False)
    return(point_df)

def point_bund(df):
    n = list(df["Команда1"].value_counts().index)
    d = dict()
    for i in n:
        d.update({i:0})
    df = df.sort_values(by="Tour",axis=0)
    for i in list(df["id"]):
        num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
        
        df.loc[df["id"] == i ,["Point_1"]] = d[num[0]]
        df.loc[df["id"] == i ,["Point_2"]]  = d[num[2]]
        
        if num[1] > num[3]:
            d[num[0]] +=  3
        elif num[1] < num[3]:
            d[num[2]] +=  3
        else:
            d[num[0]] +=  1
            d[num[2]] +=  1
    return df

def point_rang(df):# заполняет данные очками и рангом действующия
    d = pd.DataFrame({
        "team": list(df["Команда1"].value_counts().index),
        "point": [0] * len(list(df["Команда1"].value_counts().index)),
        "rang": [0] * len(list(df["Команда1"].value_counts().index)),
        "gool": [0] * len(list(df["Команда1"].value_counts().index)),
    })

    df = df.sort_values(by="Tour",axis=0)

    tour = df.loc[df["id"] == list(df["id"])[0] ,["Tour"]].values[0][0]

    for i in list(df["id"]):
        if tour == df.loc[df["id"] == i ,["Tour"]].values[0][0]:
            num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
            #запись данных в датафрейм:
            df.loc[df["id"] == i ,["Point_1"]] = d.loc[d["team"] == num[0],["point"]].values[0][0]
            df.loc[df["id"] == i ,["Point_2"]] = d.loc[d["team"] == num[2],["point"]].values[0][0]

            df.loc[df["id"] == i ,["rang_1"]] = d.loc[d["team"] == num[0],["rang"]].values[0][0]
            df.loc[df["id"] == i ,["rang_2"]] = d.loc[d["team"] == num[2],["rang"]].values[0][0]
            
            # обновление данных в опереативной таблице:
            if num[1] > num[3]:
                d.loc[d["team"] == num[0],["point"]] +=  3
            elif num[1] < num[3]:
                d.loc[d["team"] == num[2],["point"]] +=  3
            else:
                d.loc[d["team"] == num[0],["point"]] +=  1
                d.loc[d["team"] == num[2],["point"]] +=  1
                
            d.loc[d["team"] == num[0],["gool"]] += num[1]
            d.loc[d["team"] == num[2],["gool"]] += num[3]
            
        else:
            #перезаписование тура
            tour = df.loc[df["id"] == i ,["Tour"]].values[0][0]
            #сортировка оперативной таблици, перезапись рангов
            d = d.sort_values(by=["point","gool"],ascending=[False,False])
            d.loc[:,"rang"] = range(1, len(list(df["Команда1"].value_counts().index))+1)
            
            num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
            #запись данных в датафрейм:
            df.loc[df["id"] == i ,["Point_1"]] = d.loc[d["team"] == num[0],["point"]].values[0][0]
            df.loc[df["id"] == i ,["Point_2"]] = d.loc[d["team"] == num[2],["point"]].values[0][0]

            df.loc[df["id"] == i ,["rang_1"]] = d.loc[d["team"] == num[0],["rang"]].values[0][0]
            df.loc[df["id"] == i ,["rang_2"]] = d.loc[d["team"] == num[2],["rang"]].values[0][0]
            
            # обновление данных в опереативной таблице:
            if num[1] > num[3]:
                d.loc[d["team"] == num[0],["point"]] +=  3
            elif num[1] < num[3]:
                d.loc[d["team"] == num[2],["point"]] +=  3
            else:
                d.loc[d["team"] == num[0],["point"]] +=  1
                d.loc[d["team"] == num[2],["point"]] +=  1
                
            d.loc[d["team"] == num[0],["gool"]] += num[1]
            d.loc[d["team"] == num[2],["gool"]] += num[3]
    d = d.sort_values(by=["point","gool"],ascending=[False,False])
    d.loc[:,"rang"] = range(1, len(list(df["Команда1"].value_counts().index))+1)    
    return df, d

def preobr(df):
    if 'Unnamed: 0.1' in list(df.columns):
        df = df.drop('Unnamed: 0.1',axis=1)
    if 'Unnamed: 0' in list(df.columns):
        df = df.drop('Unnamed: 0',axis=1)
    
    df = df.drop('Матчи 1',axis=1)
    df = df.drop('Матчи 2',axis=1)
    df = df.astype({'счет1': 'int32'})
    df = df.astype({'счет2': 'int32'})

    df["тотал"] = df["счет1"] + df["счет2"]
    df["Дата"] = pd.to_datetime(df['Дата'],dayfirst=True)
    df['Температура'] = df['Температура'].apply(lambda x: int(x[1:-2]))
    
    for i in [14919463, 14898101]:
        df.loc[df["id"] == i ,['Офсайды  (соперник) 2']] = 4.00
        df.loc[df["id"] == i ,['Офсайды 2']] = 1.00
    
    #mask_1 = df["id"] == 14898101
    #df['Офсайды  (соперник) 2'][mask_1] = 4.00
    #df['Офсайды 2'][mask_1] = 1.00
    
    df["rang_1"] = 0
    df["rang_2"] = 0  
    df["Point_1"] = 0
    df["Point_2"] = 0
    df["liga"] = df["общая информация"].apply(lambda x: x.split(",")[0])
    df["Tour"] = df["общая информация"].apply(lambda x: x.split(",")[1])
    
    df = df[df["Tour"] != " стыковые матчи"]
    df = df[df["Tour"] != " понижение/повышение - финал"]
    df = df[df["Tour"] != " дополнительный матч"]
    df["Tour"] = df["Tour"].apply(lambda x: int(x.strip()[:-6]))
    
    df = df.astype({'Офсайды  (соперник) 2': 'float32'})
    df = df.astype({'Офсайды 2': 'float32'})

    df["Сухие матчи 1"] = df["Сухие матчи 1"].apply(lambda x: int(x.split()[-1]))
    df["Сухие матчи 2"] = df["Сухие матчи 2"].apply(lambda x: int(x.split()[0]))

    df["обе забьют 1"] = df["обе забьют 1"].apply(lambda x: int(x.split()[-1]))
    df["обе забьют 2"] = df["обе забьют 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал больше 2.5 1"] = df["Тотал больше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал больше 2.5 2"] = df["Тотал больше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал меньше 2.5 1"] = df["Тотал меньше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал меньше 2.5 2"] = df["Тотал меньше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Поражения 1"] = df["Поражения 1"].apply(lambda x: int(x.split()[-1]))
    df["Поражения 2"] = df["Поражения 2"].apply(lambda x: int(x.split()[0]))

    df["Ничьи 1"] = df["Ничьи 1"].apply(lambda x: int(x.split()[-1]))
    df["Ничьи 2"] = df["Ничьи 2"].apply(lambda x: int(x.split()[0]))

    df["Победы 1"] = df["Победы 1"].apply(lambda x: int(x.split()[-1]))
    df["Победы 2"] = df["Победы 2"].apply(lambda x: int(x.split()[0]))

    df["Отдых: дни (часы) 1"] = df["Отдых: дни (часы) 1"].apply(lambda x: float(x.split()[-1]))
    df["Отдых: дни (часы) 2"] = df["Отдых: дни (часы) 2"].apply(lambda x: float(x.split()[0]))
    
    df, d = point_rang(df)
    return df

In [49]:
Bundes_liga_19_20 = pd.read_csv("data_auto/Bundes_lige2019-2020.csv")
Bundes_liga_20_21 = pd.read_csv("data_auto/Bundes_lige2020-2021.csv")
Bundes_liga_21_22 = pd.read_csv("data_auto/Bundes_lige2021-2022.csv")
Bundes_liga_22_23 = pd.read_csv("data_auto/Bundes_lige2022-2023.csv")

Premier_liga19_20 = pd.read_csv("data_auto/Premier_liga2019-2020.csv")
Premier_liga20_21 = pd.read_csv("data_auto/Premier_liga2020-2021.csv")
Premier_liga21_22 = pd.read_csv("data_auto/Premier_liga2021-2022.csv")
Premier_liga22_23 = pd.read_csv("data_auto/Premier_liga2022-2023.csv")

Seria_A_liga19_20 = pd.read_csv("data_auto/Seria_A_liga2019-2020.csv")
Seria_A_liga20_21 = pd.read_csv("data_auto/Seria_A_liga2020-2021.csv")
Seria_A_liga21_22 = pd.read_csv("data_auto/Seria_A_liga2021-2022.csv")
Seria_A_liga22_23 = pd.read_csv("data_auto/Seria_A_liga2022-2023.csv")

Primera_liga19_20 = pd.read_csv("data_auto/Primera_liga2019-2020.csv")
Primera_liga20_21 = pd.read_csv("data_auto/Primera_liga2020-2021.csv")
Primera_liga21_22 = pd.read_csv("data_auto/Primera_liga2021-2022.csv")
Primera_liga22_23 = pd.read_csv("data_auto/Primera_liga2022-2023.csv")

In [50]:
Bundes_liga_19_20_cl = preobr(Bundes_liga_19_20)
Bundes_liga_20_21_cl = preobr(Bundes_liga_20_21)
Bundes_liga_21_22_cl = preobr(Bundes_liga_21_22)
Bundes_liga_22_23_cl = preobr(Bundes_liga_22_23)

Premier_liga19_20_cl = preobr(Premier_liga19_20)
Premier_liga20_21_cl = preobr(Premier_liga20_21)
Premier_liga21_22_cl = preobr(Premier_liga21_22)
Premier_liga22_23_cl = preobr(Premier_liga22_23)

Seria_A_liga19_20_cl = preobr(Seria_A_liga19_20)
Seria_A_liga20_21_cl = preobr(Seria_A_liga20_21)
Seria_A_liga21_22_cl = preobr(Seria_A_liga21_22)
Seria_A_liga22_23_cl = preobr(Seria_A_liga22_23)

Primera_liga19_20_cl = preobr(Primera_liga19_20)
Primera_liga20_21_cl = preobr(Primera_liga20_21)
Primera_liga21_22_cl = preobr(Primera_liga21_22)
Primera_liga22_23_cl = preobr(Primera_liga22_23)

In [51]:
df = pd.concat([Bundes_liga_19_20_cl,
               Bundes_liga_20_21_cl,
               Bundes_liga_21_22_cl,
               Bundes_liga_22_23_cl,
               Premier_liga19_20_cl,
               Premier_liga20_21_cl,
               Premier_liga21_22_cl,
               Premier_liga22_23_cl,
               Seria_A_liga19_20_cl,
               Seria_A_liga20_21_cl,
               Seria_A_liga21_22_cl,
               Seria_A_liga22_23_cl,
               Primera_liga19_20_cl,
               Primera_liga20_21_cl,
               Primera_liga21_22_cl,
               Primera_liga22_23_cl])


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5784 entries, 307 to 0
Data columns (total 77 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              5784 non-null   int64         
 1   общая информация                5784 non-null   object        
 2   Дата                            5784 non-null   datetime64[ns]
 3   Команда1                        5784 non-null   object        
 4   счет1                           5784 non-null   int32         
 5   Команда2                        5784 non-null   object        
 6   счет2                           5784 non-null   int32         
 7   Температура                     5784 non-null   int64         
 8   Признак погоды                  5784 non-null   object        
 9   поб1                            5784 non-null   float64       
 10  X                               5784 non-null   float64       
 11  поб2 

In [82]:
df["month"] = df["Дата"].to_period(freq='M')



fig = px.line(df.pivot_table(
    values='тотал',
    index=["month"],
    columns='liga',
    aggfunc="mean",
    fill_value=0).values)
fig.show()

TypeError: unsupported Type Int64Index

In [75]:
a = df.pivot_table(
    values='тотал',
    index=['year',"month"],
    columns='liga',
    aggfunc="mean",
    fill_value=0)
a.columns = a.columns.map(''.join)
a.

liga        Бундеслига  Премьер-Лига   Примера   Серия А
year month                                              
2019 8        3.400000      2.736842  2.269231  3.230769
     9        3.137931      3.156250  2.590909  2.765957
     10       2.814815      2.700000  2.564103  2.897436
     11       3.606061      2.916667  2.742857  2.636364
     12       3.102564      2.666667  2.777778  3.189189
2020 1        3.368421      2.609756  2.133333  2.700000
     2        3.475000      2.555556  2.590909  3.076923
     3        2.750000      2.166667  2.375000  3.500000
     5        3.114286      0.000000  0.000000  0.000000
     6        3.085106      2.153846  2.415094  3.538462
     7        0.000000      2.939394  2.263158  3.227273
     8        0.000000      0.000000  0.000000  3.200000
     9        3.500000      3.678571  2.433333  3.750000
     10       2.757576      2.852941  2.052632  3.774194
     11       3.733333      2.500000  2.800000  2.605263
     12       2.861111      2.526316  2.365385  2.959184
2021 1        2.962963      2.400000  2.734694  2.885246
     2        2.857143      2.519231  2.534884  2.512821
     3        2.785714      2.242424  2.424242  3.179487
     4        2.857143      2.897436  2.509804  3.250000
     5        3.366667      2.980392  2.653061  3.137255
     8        3.185185      2.833333  2.100000  3.300000
     9        2.777778      2.433333  2.605263  3.200000
     10       3.250000      3.102564  2.413043  2.897959
     11       2.888889      2.593750  2.656250  2.828571
     12       3.250000      3.000000  2.777778  3.088889
2022 1        3.222222      2.806452  2.457143  3.027778
     2        3.333333      2.500000  2.780488  2.500000
     3        2.538462      2.812500  2.290323  2.100000
     4        3.113636      2.680000  2.500000  2.833333
     5        3.550000      3.261905  2.377778  2.891892
     8        3.055556      2.897959  2.333333  2.315789
     9        2.464286      3.111111  3.000000  2.562500
     10       3.659091      2.627119  2.508475  2.800000
     11       3.370370      3.300000  2.250000  2.566667
     12       0.000000      2.944444  2.100000  0.000000
2023 1        3.629630      2.257143  1.842105  2.700000
     2        2.888889      2.325000  2.738095  2.275000
     3        2.857143      2.967742  2.258065  2.266667
     4        2.931818      3.306452  2.807018  2.360000
     5        3.583333      2.979167  2.596154  2.880000
     6        0.000000      0.000000  2.800000  3.100000

In [81]:
ts = pd.Timestamp('2020-03-14T15:32:52.192548651')
ts.to_period(freq='M')
Period('2020-03', 'M')

Timestamp('2020-03-15 00:00:00')